In [ ]:
import panel as pn
pn.extension('vega')

The ``Vega`` pane renders Vega-based plots (including those from Altair) inside a panel. It optimizes the plot rendering by using binary serialization for any array data found on the Vega/Altair object, providing huge speedups over the standard JSON serialization employed by Vega natively. Note that to use the ``Vega`` pane in the notebook the Panel extension has to be loaded with 'vega' as an argument to ensure that vega.js is initialized.

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``debounce``** (int or dict): The debounce timeout to apply to selection events, either specified as a single integer value (in milliseconds) or a dictionary that declares a debounce value per event. Debouncing ensures that events are only dispatched N milliseconds after a user is done interacting with the plot.
* **``object``** (dict or altair Chart): Either a dictionary containing a Vega or Vega-Lite plot specification, or an Altair Chart
* **``theme``** (str): A theme to apply to the plot, must be one of 'excel', 'ggplot2', 'quartz', 'vox', 'fivethirtyeight', 'dark', 'latimes', 'urbaninstitute', 'googlecharts'.
* **``show_actions``** (boolean): Whether to show chart actions menu such as save, edit etc.

Readonly parameters:

* **``selection``** (Selection): The Selection object exposes parameters which reflect the selections declared on the plot into Python. 

___

The ``Vega`` pane supports both ``vega`` and ``vega-lite`` specs which may be provided in a raw form (i.e. a dictionary) or by defining an ``altair`` plot.

To display ``vega`` and ``vega-lite`` specification simply construct a ``Vega`` pane directly or pass it to ``pn.panel``:

In [ ]:
vegalite = {
  "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/barley.json"},
  "mark": "bar",
  "encoding": {
    "x": {"aggregate": "sum", "field": "yield", "type": "quantitative"},
    "y": {"field": "variety", "type": "nominal"},
    "color": {"field": "site", "type": "nominal"}
  }
}
vgl_pane = pn.panel(vegalite, height=240)
vgl_pane

Like all other panes, the ``Vega`` pane ``object`` can be updated, either in place and triggering an update:

In [ ]:
vegalite['mark'] = 'area'
vgl_pane.param.trigger('object')

or by replacing the ``object`` entirely:

In [ ]:
vgl_pane.object = {
  "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
  "data": {
    "url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/disasters.csv"
  },
  "width": 600,
  "height": 400,
  "transform": [
    {"filter": "datum.Entity !== 'All natural disasters'"}
  ],
  "mark": {
    "type": "circle",
    "opacity": 0.8,
    "stroke": "black",
    "strokeWidth": 1
  },
  "encoding": {
    "x": {
      "field": "Year",
      "type": "nominal",
      "axis": {"labelAngle": 90}
    },
    "y": {"field": "Entity", "type": "nominal", "axis": {"title": ""}},
    "size": {
      "field": "Deaths",
      "type": "quantitative",
      "legend": {"title": "Annual Global Deaths", "clipHeight": 30},
      "scale": {"range": [0, 5000]}
    },
    "color": {"field": "Entity", "type": "nominal", "legend": None}
  }
}

The `vega-lite` specification can also be responsively sized by declaring the width or height to match the container:

In [ ]:
responsive_spec = dict(vgl_pane.object, width='container')

vgl_responsive_pane = pn.pane.Vega(responsive_spec)
vgl_responsive_pane

Please note that the `vega` specification does not support setting `width` and `height` to `container`.

### Altair

A more convenient way of defining a Vega chart is to declare it using [altair](https://altair-viz.github.io), which provides a declarative API on top of vega-lite. The ``Vega`` pane will automatically render the Vega-Lite spec when passed an Altair chart:

In [ ]:
import altair as alt
from vega_datasets import data

cars = data.cars()

chart = alt.Chart(cars).mark_circle(size=60).encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
).interactive()

altair_pane = pn.panel(chart)
altair_pane

The Altair chart can also be updated by updating the pane ``object``:

In [ ]:
altair_pane.object = chart.mark_circle(size=100)

All the usual layouts and composition operators that Altair supports can also be rendered:

In [ ]:
penguins_url = "https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json"

chart1 = alt.Chart(penguins_url).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color='Species:N'
).properties(
    height=300,
    width=300,
)

chart2 = alt.Chart(penguins_url).mark_bar().encode(
    x='count()',
    y=alt.Y('Beak Depth (mm):Q', bin=alt.Bin(maxbins=30)),
    color='Species:N'
).properties(
    height=300,
    width=100
)

pn.panel(chart1 | chart2)

### Selections

The `Vega` pane automatically syncs any selections expressed on the Vega/Altair chart. As an example we can add an Altair `selection_interval` selection to our chart:

In [ ]:
brush = alt.selection_interval(name='brush')  # selection of type "interval"

chart = alt.Chart(penguins_url).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_selection(
    brush
)

vega_pane = pn.pane.Vega(chart, debounce=10)

vega_pane

Note we specified a single `debounce` value, if we declare multiple selections we can instead declare a debounce value per named event by specifying it as a dictionary, e.g. `debounce={'brush': 10, ...}`.

The named selection will now appear on the `.selection` sub-object:

In [ ]:
vega_pane.selection

By inspecting the JSON representation of the Altair chart we can see how to express these selections in vega(-lite):

In [ ]:
chart.to_dict()['selection']

For more background see the [Altair](https://altair-viz.github.io/user_guide/interactions.html) documentation on available interactions.

As a final step we're going to create an example that binds the `brush` selection to a function which returns a Column containing the current query and the currently selected data:

In [ ]:
import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json")

brush = alt.selection_interval(name='brush')

vega_pane = pn.pane.Vega(chart.add_selection(
    brush
))

def filtered_table(selection):
    if selection is None:
        return '## No selection'
    query = ' & '.join(
        f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}'
        for col, crange in selection.items()
    )
    return pn.Column(
        f'Query: {query}',
        pn.pane.DataFrame(df.query(query), width=600, height=300)
    )

pn.Row(vega_pane, pn.bind(filtered_table, vega_pane.selection.param.brush))

### Controls

The `Vega` pane exposes a number of options which can be changed from both Python and Javascript. Try out the effect of these parameters interactively:

In [ ]:
pn.Row(vgl_responsive_pane.controls(jslink=True), vgl_responsive_pane, sizing_mode="stretch_width")